# Zonal statistics <img align="right" src="../img/LivingWales_logo.png" width="190" height="200">

* **Compatibility:** Notebook currently compatible with the `WDC` environment
* **Products used:** `sen2_l2a_gcp`
* **Special requirement:** 
A shapefile containing the polygon you would like to use for the analysis.

## Description
In the Open Data Cube, a polygon can be used to grab a stack of imagery that corresponds to the location of an input polygon. 
It is a useful tool for generating zonal statistics.

This notebook shows you how to:

1. Upload a file from your computer
2. Use a polygon's geometry to generate a `dc.load` query
3. Mask the returned data with the polygon geometry (to remove pixels outside the polygon)
4. Calculate and plot zonal statistics

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

In [ ]:
import datacube
import geopandas as gpd

from datacube.utils.geometry import Geometry
from ipyleaflet import GeoData

import sys

sys.path.append("../wales_utils/data_cube_utilities")
from display_tools import map_geom, rgb
from wdc_datahandling import cleaning_s2, cloud_coverage, geopolygon_masking
from wdc_bandindices import calculate_indices

### Connect to the datacube

Connect to the datacube so we can access WDC data.
The `app` parameter is a unique name used to identify the notebook that does not have any effect on the analysis.

In [ ]:
dc = datacube.Datacube(app="Polygon")

## Load up the shapefile you want to use for the analysis

### Upload a file from your computer
Sometimes, you will want to import some files in your `WDC` space for an analysis.

To upload a file from your computer, open the directory browser, drag your file in the `uploads` directory and drop it (when a <span style="display:inline-block;heigth:20px;width:20px;border-radius:10px;background-color:#50b745;color:white;text-align:center">+</span> symbol appears).

> **Before running the next cell:** upload the shapefile (i.e., several files) you would like to use in the `uploads` directory and indicate the filename of your shapefile in the cell.

There is an example shapefile (`WDC_workshop.shp`) provided in the `vectors` directory if you don't have your own to use.

### Load shapefile

In [ ]:
# Open and read the shapefile
shapefile = gpd.read_file("../uploads/WDC_workshop.shp")

### Visualise shapefile

In [ ]:
# Check that the polygon loaded as expected. We'll just print the first 3 rows to check
shapefile.head(3)

### Map shapefile

The next cell transform the shapefile into geographic data and map the geometry.
The custom `map_geom()` function, available in the `WDC`, allows to visualise your shapefile on an interactive map (i.e., Open Street map and ESRI World Imagery).

World Imagery provides very high resolution (one meter or better) satellite and aerial imagery in many parts of the world. For Wales, images are Maxar (Vivid) imagery, with a 0.50 meters resolution. 

In [ ]:
# Transform shapefile boundaries into geographic data (and affect a style)
geo_data = GeoData(
    geo_dataframe=shapefile.to_crs(epsg=4326),
    style={
        "color": "black",
        "fillColor": "#3366cc",
        "opacity": 0.05,
        "weight": 1.9,
        "dashArray": "2",
        "fillOpacity": 0.6,
    },
    hover_style={"fillColor": "red", "fillOpacity": 0.2},
    name="Shapefile",
)

# map the geographic data on dynamic map
m = map_geom(geo_data)
m

## Query the datacube using the polygon we have loaded

### Set up the `dc.load` query

We need to grab the geometry from the polygon we want to use for the polygon drill. 
For this example, we'll just grab the first polygon feature from the file using `.iloc[0]`:

In [ ]:
geom = Geometry(geom=shapefile.iloc[0].geometry, crs=shapefile.crs)
geom

To set up the query, we need to set up several additional parameters:

- `geopolygon`: Here we input the geometry we want to use for the drill that we prepared in the cell above
- `time`: The temporal extent. The time dimension can be specified using a tuple of datetime objects or strings in the "YYYY", "YYYY-MM" or "YYYY-MM-DD" format. 
- `output_crs`: The output projection system in EPSG:code
- `resolution`: The output spatial resolution with `output_crs` unit (e.g., in meters for EPSG:27700)

- `measurements`: Here is where you specify which bands you want to extract. 
We will just be plotting a true colour image and calculating NDVI, so we just need RGB and `nir` (and the `scl` cloud layer)

In [ ]:
query = {
    "geopolygon": geom,
    "time": ("2020-04-01", "2020-07-31"),
    "output_crs": "EPSG:27700",
    "resolution": (-10, 10),
    "measurements": ["red", "green", "blue", "nir", "scl"],
    "dask_chunks": {"y": 2048, "x": 2048},
}

### Use the query to extract data

In [ ]:
# Load data for our polygon and time period
dataset = dc.load(product="sen2_l2a_gcp", **query)

# Clean the Sentinel-2 dataset (i.e., cloud masking and reflectance normalisation)
dataset_clean = cleaning_s2(dataset)

# Calculating the cloud coverage (%) for each date
cloud_percentage = cloud_coverage(dataset_clean)
# Dropping dates where cloud percentage greater than cloud maximum threshold
cloud_mask = cloud_percentage <= 20
cloud_mask = cloud_mask.compute()
dataset_2use = dataset_clean.where(cloud_mask, drop=True)

# Check we have some data back with multiple timesteps
dataset_2use

In [ ]:
rgb(dataset_2use, col="time", col_wrap=6, robust=True)

## Mask data using the  geopolygon
The data returned from our polygon drill contains data for the bounding box of the extents of the input polygon, not the actual shape of the polygon. 

To get rid of the bits of the image located outside the polygon, we need to mask the data using the geopolygon.

In this section, we will mask out the pixels which are not included within the selected shapefile feature using one of the tools developed by Living Wales: `geopolygon_masking()`

In [ ]:
dataset_masked = geopolygon_masking(dataset_2use, geopolygon=geom)

rgb(dataset_masked, col="time", col_wrap=6, robust=True)

## Calculate zonal statistics

It can be useful to calculate some zonal statistics for identifying some evolution in a plot.

For example, tracking the average NDVI index at parcel scale can help monitoring the evolution of vegetation growth within a field plot.

In [ ]:
# Start by calculating NDVI
dataset_NDVI = calculate_indices(
    dataset_masked, index="NDVI", platform="SENTINEL_2", drop=True, quiet=True
)
dataset_NDVI

To generate the average NDVI at parcel scale, we use the `xarray.mean` method, specifying `['x','y']` as the dimension to compute the average over.

In [ ]:
# Then calculate the average NDVI value at parcel scale
average_parcel_NDVI = dataset_NDVI.NDVI.mean(["x", "y"])

In [ ]:
# Plot the average parcel NDVI
average_parcel_NDVI.plot()

## Recommended next steps

For more advanced information about working with Jupyter Notebooks or JupyterLab, see the [JupyterLab documentation](https://jupyterlab.readthedocs.io/en/stable/user/notebook.html).

To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. **[Introduction to jupyter Notebooks](01_Introduction_jupyter_notebooks.ipynb)**
2. **[Products and measurements](02_Products_and_measurements.ipynb)**
3. **[Loading data in WDC](03_Loading_data.ipynb)**
4. **[Plotting](04_Plotting.ipynb)**
5. **[Using Sentinel-2 data](05_Using_Sentinel2_data.ipynb)**
6. **[Calculating band indices](06_Calculating_band_indices.ipynb)**
7. **[Generating composites](07_Generating_composites.ipynb)**
8. **Zonal_statistics (this notebook)**


Once you have worked through the beginner's guide, you can explore the "Case Studies" directory, which provides examples of applications within Wales Open Data Cube.